# Get Votes

Récupérer les votes des parlementaires actuels en JSON grâce à l’API du parlement

http://ws-old.parlament.ch/

«Pour le moment, les nouveaux services web ne sont pas encore disponibles. Nous sommes cependant certains que nous pourrons remédier à cette situation d’ici au troisième trimestre 2019.»

Votes:
http://ws-old.parlament.ch/votes/councillors

Exemple de liste de votes:


http://ws-old.parlament.ch/votes/councillors/3055?legislativePeriodFiter=662

In [1]:
import pandas as pd
import json
import requests
from time import sleep
pd.set_option('display.max_colwidth', 999)

In [2]:
# df à actualiser avec 1_Get_Councillors.ipynb
# (en l’occurence: impossible d’actualiser à cause de la colonne
# "number", absente de la ressource utilisée dans 1_Get_Councillors)
df_national = pd.read_csv('data/councillors/national_all.csv')
df_etats = pd.read_csv('data/councillors/states_all.csv')

In [3]:
df_national['factionName'].value_counts()

Groupe de l'Union démocratique du Centre    81
Groupe socialiste                           55
Groupe libéral-radical                      42
Le Groupe du Centre. Le Centre. PEV.        31
Groupe des VERT-E-S                         30
Groupe vert'libéral                         19
Groupe PDC                                   9
Groupe des Verts                             4
Groupe BD                                    4
Name: factionName, dtype: int64

In [6]:
len('==========================================================================================')

90

In [9]:
90 + len('==============================================================================================================================================================================')

264

In [12]:
# Si on veut cibler un groupe précis:
# for i, row in df_national[df_national['factionName'] == 'Groupe PDC'].iterrows():

# On peut aussi filtrer avec legislativePeriodFilter=49
exception_list = []
delay = 0

for i, row in df_national[88:90].iterrows():
    print('', end='=')
    number = row['number']
    try:
        response = requests.get('http://ws-old.parlament.ch/votes/councillors/{}?format=json&lang=fr'.format(number), headers={'Accept': 'application/json', 'Accept-Language': 'fr-CH, fr;q=0.9, en;q=0.8, de;q=0.7, *;q=0.5'})
    except Exception as e:
        print(f"Erreur {e} pour {row['firstName']} {row['lastName']} (id: {row['id']})")
        continue
    try:
        df = pd.DataFrame(response.json())
    except:
        print(f"JSON invalide pour {row['firstName']} {row['lastName']} (id: {row['id']})")
        print(f"Lien:\nhttp://ws-old.parlament.ch/votes/councillors/{row['id']}?format=json&lang=fr")
        exception_list.append(row['id'])
        continue

    # On aplatit le json
    affairs = df['affairVotes'].tolist()
    df_affair = pd.DataFrame(affairs)
    df_affair['decision'] = df_affair['councillorVote'].apply(lambda x: x['decision'])
    
    dfm = df.join(df_affair, rsuffix='_')
    del dfm['affairVotes']
    del dfm['councillorVote']
    
    # On enregistre
    dfm.to_csv('data/votes/national/{}_votes.csv'.format(number))
    sleep(delay)

==

In [14]:
# Ne fonctionne que pour 29 des Conseillers d'Etat, il faudra voir pourquoi

# Si on veut cibler un groupe précis:
# for i, row in df_national[df_national['factionName'] == 'Groupe PDC'].iterrows():

for i, row in df_etats.iterrows():
    print('', end='=')
    number = row['number']
    try:
        response = requests.get('http://ws-old.parlament.ch/votes/councillors/{}?format=json&lang=fr'.format(number), headers={'Accept': 'application/json', 'Accept-Language': 'fr-CH, fr;q=0.9, en;q=0.8, de;q=0.7, *;q=0.5'})
    except Exception as e:
        print(f"Erreur {e} pour {row['firstName']} {row['lastName']} (id: {row['id']})")
        continue
        
    try:
        df = pd.DataFrame(response.json())
    except:
        print(f"JSON invalide pour {row['firstName']} {row['lastName']} (id: {row['id']})")
        print(f"Lien:\nhttp://ws-old.parlament.ch/votes/councillors/{row['id']}?format=json&lang=fr")
        exception_list.append(row['id'])
        continue
    
    affairs = df['affairVotes'].tolist()
    df_affair = pd.DataFrame(affairs)
    
    # On aplatit et enregistre
    dfm = df.join(df_affair, rsuffix='_')
    del dfm['affairVotes']
    del dfm['councillorVote']
    dfm.to_csv('data/votes/states/{}_votes.csv'.format(number))
    sleep(.5)

JSON invalide pour Elisabeth Baume-Schneider (id: 4238)
Lien:
http://ws-old.parlament.ch/votes/councillors/4238?format=json&lang=fr
JSON invalide pour Isabelle Chassot (id: 4325)
Lien:
http://ws-old.parlament.ch/votes/councillors/4325?format=json&lang=fr
JSON invalide pour Josef Dittli (id: 4151)
Lien:
http://ws-old.parlament.ch/votes/councillors/4151?format=json&lang=fr
JSON invalide pour Stefan Engler (id: 4055)
Lien:
http://ws-old.parlament.ch/votes/councillors/4055?format=json&lang=fr
JSON invalide pour Erich Ettlin (id: 4206)
Lien:
http://ws-old.parlament.ch/votes/councillors/4206?format=json&lang=fr
JSON invalide pour Johanna Gapany (id: 4306)
Lien:
http://ws-old.parlament.ch/votes/councillors/4306?format=json&lang=fr
JSON invalide pour Hannes Germann (id: 825)
Lien:
http://ws-old.parlament.ch/votes/councillors/825?format=json&lang=fr
JSON invalide pour Thomas Hefti (id: 4139)
Lien:
http://ws-old.parlament.ch/votes/councillors/4139?format=json&lang=fr
JSON invalide pour Peter Heg

### La liste des détails par vote est ici
Ceux du CdE dont le JSON ne fonctionne pas ne sont pas listés

http://ws-old.parlament.ch/votes/councillors